In [94]:
import sys

sys.path.append("..")

In [95]:
from util.run import solve

In [96]:
domain = """
agent(alice;bob).
time(0..8).
amount(0..100).
commodity("EUR";"good").
action(payer_payee_amount_commodity(Payer, Payee, Amount, Commodity)) :-
  agent(Payer), agent(Payee),
  amount(Amount),
  commodity(Commodity).
"""
solve(domain);

Answer  1: { action(payer_payee_amount_commodity(alice,alice,0,"EUR")) action(payer_payee_amount_commodity(alice,alice,0,"good")) action(payer_payee_amount_commodity(alice,alice,1,"EUR")) action(payer_payee_amount_commodity(alice,alice,1,"good")) action(payer_payee_amount_commodity(alice,alice,2,"EUR")) action(payer_payee_amount_commodity(alice,alice,2,"good")) action(payer_payee_amount_commodity(alice,alice,3,"EUR")) action(payer_payee_amount_commodity(alice,alice,3,"good")) action(payer_payee_amount_commodity(alice,alice,4,"EUR")) action(payer_payee_amount_commodity(alice,alice,4,"good")) action(payer_payee_amount_commodity(alice,alice,5,"EUR")) action(payer_payee_amount_commodity(alice,alice,5,"good")) action(payer_payee_amount_commodity(alice,alice,6,"EUR")) action(payer_payee_amount_commodity(alice,alice,6,"good")) action(payer_payee_amount_commodity(alice,alice,7,"EUR")) action(payer_payee_amount_commodity(alice,alice,7,"good")) action(payer_payee_amount_commodity(alice,alice,8,"

In [97]:
obs = """
obs(agent_account_balance_commodity(alice, register, 100, "EUR"), 0).
obs(agent_account_balance_commodity(alice, store, 0, "good"), 0).
obs(agent_account_balance_commodity(bob, register, 0, "EUR"), 0).
obs(agent_account_balance_commodity(bob, store, 1, "good"), 0).


obs(agent_account_balance_commodity(alice, commitments, 0, Obligation), 0) :-
  Payment = payer_payee_amount_commodity(alice, bob, 100, "EUR"),
  Obligation = debtor_creditor_action_start_end(alice, bob, Payment, 2, 8).

obs(agent_account_balance_commodity(bob, demands, 0, Obligation), 0) :-
  Payment = payer_payee_amount_commodity(alice, bob, 100, "EUR"),
  Obligation = debtor_creditor_action_start_end(alice, bob, Payment, 2, 8).

obs(agent_account_balance_commodity(bob, commitments, 0, Obligation), 0) :-
  Payment = payer_payee_amount_commodity(bob, alice, 1, "good"),
  Obligation = debtor_creditor_action_start_end(bob, alice, Payment, 2, 8).

obs(agent_account_balance_commodity(alice, demands, 0, Obligation), 0) :-
  Payment = payer_payee_amount_commodity(bob, alice, 1, "good"),
  Obligation = debtor_creditor_action_start_end(bob, alice, Payment, 2, 8).

"""
solve(obs);

Answer  1: { obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),2,8)),0) obs(agent_account_balance_commodity(alice,demands,0,debtor_creditor_action_start_end(bob,alice,payer_payee_amount_commodity(bob,alice,1,"good"),2,8)),0) obs(agent_account_balance_commodity(alice,register,100,"EUR"),0) obs(agent_account_balance_commodity(alice,store,0,"good"),0) obs(agent_account_balance_commodity(bob,commitments,0,debtor_creditor_action_start_end(bob,alice,payer_payee_amount_commodity(bob,alice,1,"good"),2,8)),0) obs(agent_account_balance_commodity(bob,demands,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),2,8)),0) obs(agent_account_balance_commodity(bob,register,0,"EUR"),0) obs(agent_account_balance_commodity(bob,store,1,"good"),0) }.
SAT 1 


In [98]:
occs = """

occ(contract_agreement, 1).
occ(payer_payee_amount_commodity(alice, bob, 100, "EUR"), 3).
%occ(payer_payee_amount_commodity(bob, alice, 1, "good"), 6).

"""
solve(occs);

Answer  1: { occ(contract_agreement,1) occ(payer_payee_amount_commodity(alice,bob,100,"EUR"),3) }.
SAT 1 


In [99]:
contract_rules = """

occ(Obligation, T) :-
  Payment = payer_payee_amount_commodity(alice, bob, 100, "EUR"),
  Obligation = debtor_creditor_action_start_end(alice, bob, Payment, T+1, T+7),
  occ(contract_agreement, T).

occ(Obligation, T) :-
  Payment = payer_payee_amount_commodity(bob, alice, 1, "good"),
  Obligation = debtor_creditor_action_start_end(bob, alice, Payment, T+1, T+7),
  occ(contract_agreement, T).

"""
solve(contract_rules);

Answer  1: {  }.
SAT 1 


In [100]:
obligation_rules = """

occ(credit(Debtor, commitments, 1, Obligation), T) :-
  Obligation = debtor_creditor_action_start_end(Debtor, Creditor, Action, Start, End),
  occ(Obligation, T).

occ(debit(Creditor, demands, 1, Obligation), T) :-
  Obligation = debtor_creditor_action_start_end(Debtor, Creditor, Action, Start, End),
  occ(Obligation, T).

"""
solve(obligation_rules);

Answer  1: {  }.
SAT 1 


In [101]:

obligation_fulfilment_rules = """

occ(debit(Debtor, commitments, 1, Obligation), T) :-
    occ(Action, T),
    action(Action),
    obs(agent_account_balance_commodity(Debtor, commitments, C, Obligation), T),
    C >= 1,
    Obligation = debtor_creditor_action_start_end(Debtor, Creditor, Action, Start, End),
    time(T), time(Start), time(End), Start <= T, T <= End,
    agent(Debtor), agent(Creditor).

occ(credit(Creditor, demands, 1, Obligation), T) :-
    occ(Action, T),
    action(Action),
    obs(agent_account_balance_commodity(Debtor, commitments, C, Obligation), T),
    C >= 1,
    Obligation = debtor_creditor_action_start_end(Debtor, Creditor, Action, Start, End),
    time(T), time(Start), time(End), Start <= T, T <= End,
    agent(Debtor), agent(Creditor).


"""
solve(obligation_fulfilment_rules);

Answer  1: {  }.
SAT 1 


<block>:5:5-19: info: atom does not occur in any rule head:
  action(Action)

<block>:6:5-80: info: atom does not occur in any rule head:
  obs(agent_account_balance_commodity(Debtor,commitments,C,Obligation),T)

<block>:9:5-12: info: atom does not occur in any rule head:
  time(T)

<block>:9:14-25: info: atom does not occur in any rule head:
  time(Start)

<block>:9:27-36: info: atom does not occur in any rule head:
  time(End)

<block>:10:5-18: info: atom does not occur in any rule head:
  agent(Debtor)

<block>:10:20-35: info: atom does not occur in any rule head:
  agent(Creditor)

<block>:14:5-19: info: atom does not occur in any rule head:
  action(Action)

<block>:15:5-80: info: atom does not occur in any rule head:
  obs(agent_account_balance_commodity(Debtor,commitments,C,Obligation),T)

<block>:18:5-12: info: atom does not occur in any rule head:
  time(T)

<block>:18:14-25: info: atom does not occur in any rule head:
  time(Start)

<block>:18:27-36: info: atom does not occur

In [102]:
obligation_fulfilment_rules_simpler = """

occ(debit(Debtor, commitments, 1, Obligation), T) :-
  agent(Debtor), agent(Creditor),
  time(Start), time(End), time(T), Start <= T, T <= End,
  Obligation = debtor_creditor_action_start_end(Debtor,Creditor,Action,Start,End),
  action(Action),
  Action = payer_payee_amount_commodity(Debtor,Creditor,Amount,Commodity),
  commodity(Commodity),
  amount(Amount),
  occ(Action, T).

occ(credit(Creditor, demands, 1, Obligation), T) :-
  agent(Debtor), agent(Creditor),
  time(Start), time(End), time(T), Start <= T, T <= End,
  Obligation = debtor_creditor_action_start_end(Debtor,Creditor,Action,Start,End),
  action(Action),
  Action = payer_payee_amount_commodity(Debtor,Creditor,Amount,Commodity),
  commodity(Commodity),
  amount(Amount),
  occ(Action, T).

"""
solve(obligation_fulfilment_rules_simpler);

Answer  1: {  }.
SAT 1 


<block>:4:3-16: info: atom does not occur in any rule head:
  agent(Debtor)

<block>:4:18-33: info: atom does not occur in any rule head:
  agent(Creditor)

<block>:5:3-14: info: atom does not occur in any rule head:
  time(Start)

<block>:5:16-25: info: atom does not occur in any rule head:
  time(End)

<block>:5:27-34: info: atom does not occur in any rule head:
  time(T)

<block>:7:3-17: info: atom does not occur in any rule head:
  action(Action)

<block>:9:3-23: info: atom does not occur in any rule head:
  commodity(Commodity)

<block>:10:3-17: info: atom does not occur in any rule head:
  amount(Amount)

<block>:14:3-16: info: atom does not occur in any rule head:
  agent(Debtor)

<block>:14:18-33: info: atom does not occur in any rule head:
  agent(Creditor)

<block>:15:3-14: info: atom does not occur in any rule head:
  time(Start)

<block>:15:16-25: info: atom does not occur in any rule head:
  time(End)

<block>:15:27-34: info: atom does not occur in any rule head:
  time(T)

In [103]:

payment_rules = """

occ(debit(Payee, register, Amount, Commodity), T) :-
  occ(Payment, T),
  Payment = payer_payee_amount_commodity(Payer, Payee, Amount, Commodity),
  RAmount >= Amount,
  obs(agent_account_balance_commodity(Payer, register, RAmount, Commodity), T).

occ(debit(Payee, store, Amount, Commodity), T) :-
  occ(Payment, T),
  Payment = payer_payee_amount_commodity(Payer, Payee, Amount, Commodity),
  RAmount >= Amount,
  obs(agent_account_balance_commodity(Payer, store, RAmount, Commodity), T).

occ(credit(Payer, register, Amount, Commodity), T) :-
  occ(Payment, T),
  Payment = payer_payee_amount_commodity(Payer, Payee, Amount, Commodity),
  RAmount >= Amount,
  obs(agent_account_balance_commodity(Payer, register, RAmount, Commodity), T).

occ(credit(Payer, store, Amount, Commodity), T) :-
  occ(Payment, T),
  Payment = payer_payee_amount_commodity(Payer, Payee, Amount, Commodity),
  RAmount >= Amount,
  obs(agent_account_balance_commodity(Payer, store, RAmount, Commodity), T).

"""
solve(payment_rules);

Answer  1: {  }.
SAT 1 


<block>:7:3-79: info: atom does not occur in any rule head:
  obs(agent_account_balance_commodity(Payer,register,RAmount,Commodity),T)

<block>:13:3-76: info: atom does not occur in any rule head:
  obs(agent_account_balance_commodity(Payer,store,RAmount,Commodity),T)

<block>:19:3-79: info: atom does not occur in any rule head:
  obs(agent_account_balance_commodity(Payer,register,RAmount,Commodity),T)

<block>:25:3-76: info: atom does not occur in any rule head:
  obs(agent_account_balance_commodity(Payer,store,RAmount,Commodity),T)



In [104]:

accounts = """

account_type(demands, active).
account_type(register, active).
account_type(store, active).
account_type(commitments, passive).

"""
solve(accounts);

Answer  1: { account_type(commitments,passive) account_type(demands,active) account_type(register,active) account_type(store,active) }.
SAT 1 


In [105]:

balance_inertia_rules = """

obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T+1) :-
  obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T),
  time(T),
  not occ(debit(Agent, Account, _, Commodity), T), not occ(credit(Agent, Account, _, Commodity), T).

"""
solve(balance_inertia_rules);

Answer  1: {  }.
SAT 1 


<block>:5:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:6:7-50: info: atom does not occur in any rule head:
  occ(debit(#X0,#X1,#P2,#X3),#X4)

<block>:6:56-100: info: atom does not occur in any rule head:
  occ(credit(#X0,#X1,#P2,#X3),#X4)



In [106]:

transaction_rules = """

%%% ACTIVE

% DEBIT
obs(agent_account_balance_commodity(Agent, Account, Balance + Amount, Commodity), T+1) :-
  obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T),
  occ(debit(Agent, Account, Amount, Commodity), T),
  time(T),
  account_type(Account, active).

% CREDIT
obs(agent_account_balance_commodity(Agent, Account, Balance - Amount, Commodity), T+1) :-
  obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T),
  occ(credit(Agent, Account, Amount, Commodity), T),
  time(T),
  account_type(Account, active).

%%% PASSIVE

% DEBIT
obs(agent_account_balance_commodity(Agent, Account, Balance - Amount, Commodity), T+1) :-
  obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T),
  occ(debit(Agent, Account, Amount, Commodity), T),
  time(T),
  account_type(Account, passive).

% CREDIT
obs(agent_account_balance_commodity(Agent, Account, Balance + Amount, Commodity), T+1) :-
  obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T),
  occ(credit(Agent, Account, Amount, Commodity), T),
  time(T),
  account_type(Account, passive).

"""
solve(transaction_rules);

Answer  1: {  }.
SAT 1 


<block>:8:3-51: info: atom does not occur in any rule head:
  occ(debit(Agent,Account,Amount,Commodity),T)

<block>:9:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:10:3-32: info: atom does not occur in any rule head:
  account_type(Account,active)

<block>:15:3-52: info: atom does not occur in any rule head:
  occ(credit(Agent,Account,Amount,Commodity),T)

<block>:16:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:17:3-32: info: atom does not occur in any rule head:
  account_type(Account,active)

<block>:24:3-51: info: atom does not occur in any rule head:
  occ(debit(Agent,Account,Amount,Commodity),T)

<block>:25:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:26:3-33: info: atom does not occur in any rule head:
  account_type(Account,passive)

<block>:31:3-52: info: atom does not occur in any rule head:
  occ(credit(Agent,Account,Amount,Commodity),T)

<block>:32:3-10: info: atom does not occur in any rule head:
  time(

In [107]:
# some stupid error?
violation_rules = """

obs(violation, T) :- obs(agent_account_amount_commodity(Agent, commitments, N, _), T), time(T-1), not time(T), N > 0.
obs(violation, T) :- obs(agent_account_amount_commodity(Agent, demands, N, _), T), time(T-1), not time(T), N > 0.

"""

solve(violation_rules);

Answer  1: {  }.
SAT 1 


<block>:3:22-86: info: atom does not occur in any rule head:
  obs(agent_account_amount_commodity(#X0,commitments,#X1,#P2),#X3)

<block>:3:88-97: info: atom does not occur in any rule head:
  time((T+-1))

<block>:3:103-110: info: atom does not occur in any rule head:
  time(T)

<block>:4:22-82: info: atom does not occur in any rule head:
  obs(agent_account_amount_commodity(#X0,demands,#X1,#P2),#X3)

<block>:4:84-93: info: atom does not occur in any rule head:
  time((T+-1))

<block>:4:99-106: info: atom does not occur in any rule head:
  time(T)



In [108]:

show = """
%#show obs(agent_account_balance_commodity(alice, account, 0;1;100, Commodity), T) : time(T), commodity(Commodity).
#show obs/2.
#show occ/2.
"""

In [109]:
solve([domain,
       obs,
       occs,
       contract_rules,
       obligation_rules,
       #obligation_fulfilment_rules,
       obligation_fulfilment_rules_simpler,
       payment_rules,
       accounts,
       balance_inertia_rules,
       transaction_rules,
       violation_rules,
       show,
      ], sep='\n');

Answer  1: {
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),2,8)),0)
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),2,8)),1)
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),2,8)),4)
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),2,8)),5)
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),2,8)),6)
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),2,8)),7)
obs(agent_account_balance_commodity(ali